# ALD-01 — TinyLlama 1.1B QLoRA Fine-tuning
**Indian Bilingual LLM | English + Hindi | Colab Notebook**

---
Pipeline: Install → Dataset → 4-bit Load → LoRA → Train → Merge → GGUF → Evaluate

## 1. Install Dependencies

In [ ]:
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install -q transformers datasets peft bitsandbytes trl accelerate sentencepiece

In [ ]:
# Optional: clone your GitHub repo
# !git clone https://github.com/YOUR_USERNAME/ALD-01-LLM.git
# %cd ALD-01-LLM

import os
os.makedirs("data", exist_ok=True)
os.makedirs("outputs/ald01-lora", exist_ok=True)

## 2. Load and Prepare Dataset

In [ ]:
from datasets import load_dataset

DATASET_PATH = "data/ald01_dataset.json"

dataset = load_dataset("json", data_files=DATASET_PATH, split="train")

def format_prompt(sample):
    instruction = sample["instruction"]
    inp         = sample.get("input", "")
    output      = sample["output"]
    if inp:
        text = f"### Instruction:\n{instruction}\n\n### Input:\n{inp}\n\n### Response:\n{output}"
    else:
        text = f"### Instruction:\n{instruction}\n\n### Response:\n{output}"
    return {"text": text}

dataset = dataset.map(format_prompt)
print(f"Dataset size: {len(dataset)}")
print(dataset[0]["text"][:300])

## 3. Load Base Model in 4-bit Quantization

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit              = True,
    bnb_4bit_quant_type       = "nf4",
    bnb_4bit_compute_dtype    = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    bnb_4bit_use_double_quant = True,
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token    = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config = bnb_config,
    device_map          = "auto",
    trust_remote_code   = True,
)
model = prepare_model_for_kbit_training(model)
print("Model loaded in 4-bit.")

## 4. Configure LoRA Adapter

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r              = 16,
    lora_alpha     = 32,
    lora_dropout   = 0.05,
    target_modules = ["q_proj", "v_proj", "k_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    bias           = "none",
    task_type      = "CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()